In [524]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import datetime

In [525]:
mu_masse_1 = 5.944892676091842
s_masse_1 = 1.0452954216771342
mu_masse_2 = 4.012421689011621
s_masse_2 = 1.2811831375376432
mu_speed_1 = 8.788235294117646
s_speed_1 = 1.9745088303442118
mu_speed_2 = 37.790625
s_speed_2 = 5.31080027956004
alpha_t_delta_1 = 0.7720943018929209
beta_t_delta_1 = 32.180388904581854
alpha_t_delta_2 = 0.6924339239082432
beta_t_delta_2 = 85.33142881264439

In [526]:
def random_masse_1():
    masse_1 = np.random.lognormal(mu_masse_1, s_masse_1, 1).round(1)
    return masse_1

def random_masse_2():
    masse_2 = np.random.lognormal(mu_masse_2, s_masse_2, 1).round(1)
    return masse_2

def random_speed_1():
    speed_1 = np.random.normal(mu_speed_1, s_speed_1, 1).round(2)
    return speed_1

def random_speed_2():
    speed_2 = np.random.normal(mu_speed_2, s_speed_2, 1).round(2)
    return speed_2

def random_t_delta_1():
    t_delta_1 = np.random.exponential (scale_t_delta_1.round(0)
    return t_delta_1

def random_t_delta_1():
    t_delta_2 = np.random.exponential (scale_t_delta_2).round(0)
    return t_delta_1

In [527]:
results = []
double_entries = []



amount_of_hours = 200

for i in range (0, amount_of_hours, 1):
    results.append(i)

for i in range (0, amount_of_hours, 1):
    double_entries.append(i)
    
    
results = pd.DataFrame(results, columns= ['Hours'])
double_entries = pd.DataFrame(double_entries, columns= ['Hours'])

header = ['Day', 'Masse 1', 'Speed 1', 'Masse 2', 'Speed 2', 'Energie in kJ 1', 'Energie in kJ 2']

for value in header:
    results[value] = np.nan
    
for value in header: 
    double_entries[value] = np.nan
    

In [528]:
for value in header:
    results[value] = np.nan

In [529]:
end = amount_of_hours
start = 0
i = 0
j = 0
event_1 = 0
event_2 = 0
a = 0

In [530]:
while i < end:
    t_delta_1 = random_t_delta_1()
    if t_delta_1 > 0:
        event_1 += t_delta_1
        
        masse_1 = random_masse_1()
        results['Masse 1'][event_1] = masse_1
        
        speed_1 = random_speed_1()
        results['Speed 1'][event_1] = speed_1
     
        print("i %i" % event_1)
        i += event_1
    else: 
        masse_1 = random_masse_1()
        double_entries['Masse 1'][event_1] = masse_1
        
        speed_1 = random_speed_1()
        double_entries['Speed 1'][event_1] = speed_1
        print("i %i" % t_delta_2)

        
while j < end:
    t_delta_2 = random_t_delta_2()
    if t_delta_2 > 0:
        event_2 += t_delta_2
    
        masse_2 = random_masse_2()
        results['Masse 2'][event_2] = masse_2
    
        speed_2 = random_speed_2()
        results['Speed 2'][event_2] = speed_2
    
        print("j %i" % event_2)
    
        j += event_2
    
    else:
        masse_2 = random_masse_2()
        double_entries['Masse 2'][event_2] = masse_2
    
        speed_2 = random_speed_2()
        double_entries['Speed 2'][event_2] = speed_2 
        print("j %i" % t_delta_2)

i 28
i 50
i 70
i 71
j 1
j 2
j 3
j 4
j 5
j 6
j 7
j 8
j 9
j 10
j 11
j 12
j 13
j 14
j 15
j 16
j 17
j 18
j 19
j 20


c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\m0268176\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

In [531]:
results['Energie in kJ 1'] = results.apply(
    lambda x: results['Masse 1'] * results['Speed 1'] * results['Speed 1'] / 2 / 1000
)

results['Energie in kJ 2'] = results.apply(
    lambda x: results['Masse 2'] * results['Speed 2'] * results['Speed 2'] / 2 / 1000
)

results['Day'] = results.apply(
    lambda x: results['Hours'] // 24
)

results.fillna(0)


,Hours,Day,Masse 1,Speed 1,Masse 2,Speed 2,Energie in kJ 1,Energie in kJ 2
0,0,0,0.0,0.0,0.0,0.00,0.0,0.000000
1,1,0,0.0,0.0,46.8,28.87,0.0,19.503359
2,2,0,0.0,0.0,41.2,42.04,0.0,36.407649
3,3,0,0.0,0.0,5.6,38.89,0.0,4.234810
4,4,0,0.0,0.0,19.6,43.27,0.0,18.348470
...,...,...,...,...,...,...,...,...
195,195,8,0.0,0.0,0.0,0.00,0.0,0.000000
196,196,8,0.0,0.0,0.0,0.00,0.0,0.000000
197,197,8,0.0,0.0,0.0,0.00,0.0,0.000000
198,198,8,0.0,0.0,0.0,0.00,0.0,0.000000


In [532]:
double_entries


,Hours,Day,Masse 1,Speed 1,Masse 2,Speed 2,Energie in kJ 1,Energie in kJ 2
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
195,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,196,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,197,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,198,NaN,NaN,NaN,NaN,NaN,NaN,NaN
